### Imports 

In [17]:
import os

os.chdir('c:/Users/rsilva/Gdrive/projects/programming/automation/sei')

# HTML PARSER
from bs4 import BeautifulSoup as soup

# INITIALIZE DRIVER
from selenium import webdriver

# WAIT AND CONDITIONS METHODS
# available since 2.26.0
from selenium.webdriver.support.ui import Select

# Exceptions
from selenium.common.exceptions import TimeoutException


# METHODS
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from locators import Login, Base, LatMenu, \
    Main, ListaBlocos, Bloco, Processo, Envio


from base import Page

import re

import pandas as pd

from datetime import date

In [18]:
def login_SEI(driver, usr, pwd):
        """
        Esta função recebe um objeto Webdrive e as credenciais  
        do usuário, loga no SEI - ANATEL e retorna uma instância da classe  
        SEI. 
        """

        page = Page(driver)
        page.driver.get(Login.URL)
        page.driver.maximize_window()

        usuario = page.wait_for_element_to_click(Login.LOGIN)
        senha = page.wait_for_element_to_click(Login.SENHA)

        # Clear any clutter on the form
        usuario.clear()
        usuario.send_keys(usr)

        senha.clear()
        senha.send_keys(pwd)

        # Hit Enter
        senha.send_keys(Keys.RETURN)

        return SEI(page.driver)

In [19]:
class SEI(Page):
    """
    Esta subclasse da classe Page define métodos de execução de ações na 
    página principal do SEI e de resgate de informações
    """
    
    processos = []
    
    def ver_proc_detalhado(self):
        """
        Expands the visualization from the main page in SEI
        """
        try:
            ver_todos = self.wait_for_element_to_click(Main.FILTROATRIBUICAO)
            
            if ver_todos.text =="Ver todos os processos":
                ver_todos.click()
        
        except TimeoutException:
            
            print("A página não carregou no tempo limite ou cheque o link\
                  'ver todos os processos'")
            
        try:
            
            visual_detalhado = self.wait_for_element_to_click(Main.TIPOVISUALIZACAO)
        
            if visual_detalhado.text == "Visualização detalhada":
                visual_detalhado.click()
                
        except TimeoutException:
            
            print("A página não carregou no tempo limite ou cheque o link\
            de visualização detalhada")
            
    def isPaginaInicial(self):
        """Retorna True se a página estiver na página inicial do SEI, False
        caso contrário"""        
        return self.get_title() == 'SEI - Controle de Processos'

    def go_to_initial_page(self):
        """
        Navega até a página inicial do SEI caso já esteja nela 
        a página é recarregada
        Assume que o link está presente em qualquer subpágina do SEI
        """
        self.wait_for_element_to_click(
            Base.INITIALPAGE).click()

    def exibir_menu_lateral(self):
        """
        Exibe o Menu Lateral á Esquerda no SEI para acessos aos seus diversos
        links
        Assume que o link está presente em qualquer subpágina do SEI
        """
        menu = self.wait_for_element(Base.EXIBIRMENU)

        if menu.get_attribute("title") == "Exibir Menu do Sistema":
            menu.click()
            
    def itera_processos(self):
        """
        Navega as páginas de processos abertos no SEI e guarda as tags
        html dos processos como objeto soup no atributo processos_abertos
        """     
        
        #Apaga o conteúdo atual da lista de processos
        self.processos = []
        
        # assegura que está inicial
        if not self.isPaginaInicial():
            self.go_to_initial_page()

        # Mostra página com informações detalhadas
        self.ver_proc_detalhado()
        
        contador = Select(self.wait_for_element(Main.CONTADOR))

        pages = [pag.text for pag in contador.options]

        for pag in pages:
            
            # One simple repetition to avoid more complex code
            contador = Select(self.wait_for_element(Main.CONTADOR))
            contador.select_by_visible_text(pag)
            html_sei = soup(self.driver.page_source, "lxml")
            self.processos += html_sei("tr", {"class": 'infraTrClara'})
            
            
        # percorre a lista de processos
        # cada linha corresponde a uma tag mãe 'tr'
        # substituimos a tag mãe por uma lista das tags filhas 'tag.contents', descartando os '\n'
        # a função lista_to_dict_tags recebe essa lista e retorna um dicionário das tags
        self.processos =  [lista_to_dict_tags([tag for tag in line.contents if tag !='\n']) for line in self.processos]     

In [20]:
driver = webdriver.Chrome()
sei = login_SEI(driver, 'rsilva', 'Savorthemom3nts')

In [21]:
def lista_to_dict_tags(lista_tags):

    dict_tags = {}
    
    assert len(lista_tags) == 6, "Verifique o nº de tags de cada linha do processo, valor diferente de 10"

    #dict_tags['checkbox'] = lista_tags[0].find(class_='infraCheckbox')
    
    controles = lista_tags[1].find_all('a')    
    
    for tag_a in controles:
        
        img = str(tag_a.img['src'])
        
        if 'imagens/sei_anotacao' in img:
            
            #dict_tags['anotacao'] = tag_a
            
            dict_tags['anotacao'] = re.search('\((.*)\)', tag_a.attrs['onmouseover']).group().split("'")[1:4:2]
            
            if 'prioridade' in img: 
                
                dict_tags['prioridade'] = True
                
            else:                
                
                dict_tags['prioridade'] = False
            
        elif 'imagens/sei_situacao' in  img:
            
            dict_tags['situacao'] = re.search('\((.*)\)', tag_a.attrs['onmouseover']).group().split("'")[1]
            
        elif 'imagens/marcador' in img:
            
            #dict_tags['marcador'] = tag_a
            
            marcador = re.search('\((.*)\)', tag_a.attrs['onmouseover']).group().split("'")[1:4:2]
            
            dict_tags['marcador'] = marcador[0] + ' - ' + marcador[1]            
            
            
        elif 'imagens/exclamacao' in img:                        
           
            dict_tags['aviso'] = re.search('(\(\')(.*)(\'\))', tag_a.attrs['onmouseover']).group(2)
            
            
        peticionamento = lista_tags[1].find(src = re.compile('peticionamento'))
        
        if peticionamento:
            
            dict_tags['peticionamento'] = re.search('\((.*)\)', peticionamento.attrs['onmouseover']).group().split('"')[1]
            
            
    processo = lista_tags[2].find('a')
    
    dict_tags['processo'] = processo.string    
    
    dict_tags['visualizado'] = processo.attrs['class'][0]
    
    #dict_tags['link'] = Base.NAV_URL + processo.attrs['href']
    
    
    try:
        dict_tags['atribuicao'] = lista_tags[3].find('a').string
    
    except:        
        
        pass
                
    dict_tags['tipo'] = lista_tags[4].string
    
    try:
        dict_tags['interessado'] = lista_tags[5].find(class_='spanItemCelula').string
        
    except: 
        
        pass
        
        
    return dict_tags    


In [22]:
sei.itera_processos()

In [23]:
def dict_to_df(processos):

    tags = ['processo', 'tipo', 'visualizado', 'atribuicao',  
            'anotacao', 'prioridade', 'peticionamento', 'aviso', 'marcador', 'situacao', 'interessado']

    df = pd.DataFrame(columns=tags)

    for p in processos:

        df = df.append(pd.Series(p), ignore_index=True)
        
    df['visualizado'] = df['visualizado'].astype("category")
    df['atribuicao'] = df['atribuicao'].astype("category")
    df['tipo'] = df['tipo'].astype("category")
    df['visualizado'].cat.categories = ['NÃO', 'SIM']
    
    return df    


In [24]:
df = dict_to_df(sei.processos)

In [26]:
writer = pd.ExcelWriter('C:\\Users\\rsilva\\Google_Drive\\anatel\\processos_abertos_SEI.xlsx')
df.to_excel(writer, index=False, sheet_name = str(date.today()))
writer.save()

In [27]:
categorias = ['Outorga: Limitado Móvel Marítimo', 'Outorga: Limitado Móvel Aeronáutico',
              'Outorga: Rádio do Cidadão', 'Outorga: Radioamador']

In [7]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = ['https://spreadsheets.google.com/feeds']

credentials = ServiceAccountCredentials.from_json_keyfile_name('gspread-91fb9e2e8b49.json', scope)

gc = gspread.authorize(credentials)

AttributeError: 'Spreadsheet' object has no attribute 'close'

In [15]:
wks = gc.open(title='SEI').sheet1

In [16]:
wks.update_acell('B2', "it's down there somewhere, let me take another look.")